In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
import tensorflow as tf

data_dir='./data/flowers'

categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

data = []

In [3]:
def make_data():
    for category in categories:
      path = os.path.join(data_dir, category)
      print(path)
      label = categories.index(category)

      for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        image = cv2.imread(img_path)

        try :
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          image = cv2.resize(image, (224, 224))

          image = np.array(image, dtype=np.float32)

          data.append([image,label])

        except Exception as e:
          pass
    
    print(len(data))   

    pik = open('data.pickle', 'wb')
    pickle.dump(data,pik)
    pik.close()



In [4]:
make_data()

./data/flowers\daisy
./data/flowers\dandelion
./data/flowers\rose
./data/flowers\sunflower
./data/flowers\tulip
4317


In [5]:
def load_data():
  pick = open('data.pickle','rb')
  data = pickle.load(pick)
  pick.close()

  np.random.shuffle(data)

  feature = []
  labels = []

  for img, label in data:
    feature.append(img)
    labels.append(label)

  feature = np.array(feature, dtype=np.float32)
  labels=np.array(labels)

  feature = feature/255.0

  return [feature, labels]


In [7]:
from sklearn.model_selection import train_test_split


(feature, labels) = load_data()

(x_train, x_test, y_train, y_test) = train_test_split(feature,labels, test_size = 0.1)

categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [9]:
input_layer = tf.keras.layers.Input([224,224,3])


conv1=tf.keras.layers.Conv2D(filters = 32, kernel_size=(5,5), padding='Same',
                             activation = 'relu')(input_layer)

pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv1)
                             


conv2=tf.keras.layers.Conv2D(filters = 64, kernel_size=(3,3), padding='Same',
                             activation = 'relu')(pool1)

pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(conv2)



conv3=tf.keras.layers.Conv2D(filters = 96, kernel_size=(3,3), padding='Same',
                             activation = 'relu')(pool2)

pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv3)



conv4=tf.keras.layers.Conv2D(filters = 96, kernel_size=(3,3), padding='Same',
                             activation = 'relu')(pool3)

pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv3)


flt1 = tf.keras.layers.Flatten()(pool4)

dn1 = tf.keras.layers.Dense(512,activation='relu')(flt1)
out = tf.keras.layers.Dense(5, activation='softmax')(dn1)


In [12]:
model = tf.keras.Model(input_layer, out)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size = 100, epochs = 10)


Epoch 1/10
39/39 [==============================] - 113s 3s/step - loss: 0.6684 - accuracy: 0.7511
Epoch 2/10
39/39 [==============================] - 111s 3s/step - loss: 0.4352 - accuracy: 0.8450
Epoch 3/10
39/39 [==============================] - 112s 3s/step - loss: 0.2937 - accuracy: 0.9037
Epoch 4/10
39/39 [==============================] - 111s 3s/step - loss: 0.2158 - accuracy: 0.9277
Epoch 5/10
39/39 [==============================] - 112s 3s/step - loss: 0.1185 - accuracy: 0.9637
Epoch 6/10
39/39 [==============================] - 111s 3s/step - loss: 0.0843 - accuracy: 0.9758
Epoch 7/10
39/39 [==============================] - 111s 3s/step - loss: 0.0550 - accuracy: 0.9864
Epoch 8/10
39/39 [==============================] - 111s 3s/step - loss: 0.0366 - accuracy: 0.9918
Epoch 9/10
39/39 [==============================] - 110s 3s/step - loss: 0.0228 - accuracy: 0.9951
Epoch 10/10
39/39 [==============================] - 109s 3s/step - loss: 0.0328 - accuracy: 0.9928


In [13]:
model.save('mymodel.h5')